In [36]:
import pandas as pd
dataset_dir = "data/ml-latest-small/"
!ls $dataset_dir


original_data = pd.read_csv(dataset_dir + '/ratings.csv')
print(original_data.info())
original_data.head()

links.csv  movies.csv  ratings.csv  README.txt	tags.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [37]:
watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='watch'

clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='click'

interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [38]:
data_dir = "data"
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

In [61]:
# Build a map to convert a movie id to the movie title
movies = pd.read_csv(dataset_dir + '/movies.csv', usecols=[0,1])
movies['movieId'] = movies['movieId'].astype(str)
movie_map = dict(movies.values)
 
# Getting a random user:
user_id, item_id = interactions_df[['USER_ID', 'ITEM_ID']].sample().values[0]


In [69]:
import boto3
campaign_arn= "arn:aws:personalize:us-east-1:440010067749:campaign/personalize-demo-camp"
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)
print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']
recommendation_list = []

for item in item_list:
    title = movie_map[item['itemId']]
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
rl =recommendations_df.head()
print(rl)

uinteractions_movieslist = []
print("\n History for user: ", user_id)
 
uinteractions_df = interactions_df[interactions_df['USER_ID'] == user_id]
uinteractions_movies = uinteractions_df['ITEM_ID']

for uitem in uinteractions_movies:
    t = movie_map[item['itemId']]
    uinteractions_movieslist.append(t)
 
oldmlist = list(set(uinteractions_movieslist))

print(oldmlist)
print(uinteractions_movies.groupby('ITEM_ID'))

Recommendations for user:  310
              OriginalRecs
0  Dirty Dozen, The (1967)
1           RoboCop (1987)
2     Out of Africa (1985)
3        Goldfinger (1964)
4         High Noon (1952)

 History for user:  310
['Producers, The (1968)']


KeyError: 'ITEM_ID'

In [70]:
personalize.delete_schema(schemaArn=interaction_schema_arn)
time.sleep(60)

NameError: name 'interaction_schema_arn' is not defined